<a href="https://colab.research.google.com/github/AzulBarr/Aprendizaje-Automatico/blob/main/notebooks/notebook_09_redes_neuronales-published.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/notebooks/notebook_09_redes_neuronales-published.ipynb)

# Redes neuronales


Vamos nuevamente a trabajar con los datos de `iris` para entrenar (y antes construir) una Red Neuronal.

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_iris
import numpy as np
import matplotlib.pyplot as plt

def get_data():
    dataset = load_iris()
    X = dataset["data"]
    y = dataset["target"]
    y = LabelEncoder().fit_transform(y)
    return np.array(X), np.array(y)
X, y = get_data()
X

La propuesta es empezar por el esqueleto de las 2 clases que usaremos para esta tarea e ir implementado los métodos a medida que avancemos.

Al final de este notebook se encuentran ambas clases completas. Pueden copiar el código desde allí mismo o implementarlo. La idea es que en cada avance podamos comprender la parte del proceso que estamos realizando, por lo cual se recomienda seguir la guia propuesta e ir completando sólo lo que es necesario para cada punto.

In [ ]:
class Capa:
    def __init__(self, neuronas):
        self.neuronas = neuronas

    def forward(self, inputs, weights, bias, activation):
        """
        Forward Propagation de la capa
        """
        Z_curr = np.dot(inputs, weights.T) + bias

        if activation == 'relu':
            A_curr = self.relu(inputs=Z_curr)
        elif activation == 'softmax':
            A_curr = self.softmax(inputs=Z_curr)
        return A_curr, Z_curr

    def relu(self, inputs):
        """
        ReLU: función de activación
        """
        return np.maximum(0, inputs)

    def softmax(self, inputs):
        """
        Softmax: función de activación
        """
        exp_scores = np.exp(inputs)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        return probs

    def backward(self, dA_curr, W_curr, Z_curr, A_prev, activation):
        """
        Backward Propagation de la capa
        """
        if activation == 'softmax':
          dW = np.dot(A_prev.T, dA_curr)
          db = np.sum(dA_curr, axis=0, keepdims=True)
          dA = np.dot(dA_curr, W_curr)
        else:
          dZ = self.relu_derivative(dA_curr, Z_curr)
          dW = np.dot(A_prev.T, dZ)
          db = np.sum(dZ, axis=0, keepdims=True)
          dA = np.dot(dZ, W_curr)

        return dA, dW, db

    def relu_derivative(self, dA, Z):
        """
        ReLU: gradiente de ReLU
        """
        dZ = np.array(dA, copy = True)
        dZ[Z <= 0] = 0
        return dZ

In [ ]:
class RedNeuronal:
    def __init__(self, learning_rate=0.01):
        self.red = [] ## capas
        self.arquitectura = [] ## mapeo de entradas -> salidas
        self.pesos = [] ## W, b
        self.memoria = [] ## Z, A
        self.gradientes = [] ## dW, db
        self.lr = learning_rate

    def add(self, capa):
        """
        Agregar capa a la red
        """
        self.red.append(capa)

    def _compile(self, data):
        """
        Inicializar la arquitectura
        """
        for idx, _ in enumerate(self.red):
          if idx == 0:
            self.arquitectura.append({'input_dim': data.shape[1],
                                      'output_dim': self.red[idx].neuronas,
                                      'activation':'relu'})
          elif idx > 0 and idx < len(self.red)-1:
            self.arquitectura.append({'input_dim': self.red[idx-1].neuronas,
                                      'output_dim': self.red[idx].neuronas,
                                      'activation':'relu'})
          else:
            self.arquitectura.append({'input_dim': self.red[idx-1].neuronas,
                                    'output_dim': self.red[idx].neuronas,
                                    'activation':'softmax'})
        return self

    def _init_weights(self, data):
        """
        Inicializar arquitectura y los pesos
        """
        self._compile(data)

        np.random.seed(99)

        for i in range(len(self.arquitectura)):
            self.pesos.append({
                'W':np.random.uniform(low=-1, high=1,
                        size=(self.arquitectura[i]['input_dim'],
                            self.arquitectura[i]['output_dim']
                            )),
                'b':np.zeros((1, self.arquitectura[i]['output_dim']))})

        return self

    def _forwardprop(self, data):
        """
        Pasada forward completa por la red
        """
        A_curr = data

        for i in range(len(self.pesos)):
            A_prev = A_curr
            weights=self.pesos[i]['W'].T
            bias=self.pesos[i]['b']
            activation=self.arquitectura[i]['activation']
            A_curr, Z_curr = self.red[i].forward(inputs=A_prev,
                                                    weights=weights,
                                                    bias=bias,
                                                    activation=activation)

            self.memoria.append({'inputs':A_prev, 'Z':Z_curr})

        return A_curr

    def _backprop(self, predicted, actual):
        """
        Pasada backward completa por la red
        """
        num_samples = len(actual)

        ## compute the gradient on predictions
        dscores = predicted
        dscores[range(num_samples),actual] -= 1
        dscores /= num_samples

        dA_prev = dscores

        for idx, layer in reversed(list(enumerate(self.red))):
            dA_curr = dA_prev

            A_prev = self.memoria[idx]['inputs']
            Z_curr = self.memoria[idx]['Z']
            W_curr = self.pesos[idx]['W']

            activation = self.arquitectura[idx]['activation']

            dA_prev, dW_curr, db_curr = layer.backward(dA_curr, W_curr.T, Z_curr, A_prev, activation)

            self.gradientes.append({'dW':dW_curr, 'db':db_curr})

        self.gradientes = list(reversed(self.gradientes))  # Reverse the gradients list

    def _update(self):
        """
        Actualizar el modelo --> lr * gradiente
        """
        lr = self.lr
        for idx, layer in enumerate(self.red):
            self.pesos[idx]['W'] -= lr * self.gradientes[idx]['dW']
            self.pesos[idx]['b'] -= lr * self.gradientes[idx]['db']

    def _get_accuracy(self, predicted, actual):
        """
        Calcular accuracy después de cada iteración
        """
        return np.mean(np.argmax(predicted, axis=1)==actual)

    def _calculate_loss(self, predicted, actual):
        """
        Calcular cross-entropy loss después de cada iteración
        """
        samples = len(actual)

        correct_logprobs = -np.log(predicted[range(samples),actual])
        data_loss = np.sum(correct_logprobs)/samples

        return data_loss

    def train(self, X_train, y_train, epochs):
        """
        Entrenar el modelo Stochastic Gradient Descent
        """
        self.loss = []
        self.accuracy = []

        self._init_weights(X_train)

        for i in range(epochs):
            yhat = self._forwardprop(X_train)
            self.accuracy.append(self._get_accuracy(predicted=yhat, actual=y_train))
            self.loss.append(self._calculate_loss(predicted=yhat, actual=y_train))

            self._backprop(predicted=yhat, actual=y_train)

            self._update()

            if i % 20 == 0:
                s = 'EPOCH: {}, ACCURACY: {}, LOSS: {}'.format(i, self.accuracy[-1], self.loss[-1])
                print(s)

        return (self.accuracy, self.loss)

Los items que se presentan a continuación tienen como objetivo explorar las clases que componen la red neuronal propuesta, comprender su arquitectura y funcionamiento.

Nuevamente, lo ideal es no mirar todos los métodos hasta que llegue el momento de utilizarlos.

1. Crear una Red Neuronal con 6 nodos en la primera capa, 8 en la segunda, 10 en la tercer y finalmente 3 en la última, utilizando los métodos `add()`, `_compile()` de la clase `RedNeuronal` y el constructor de la clase `Capa`.
  
    Imprimir la arquitectura del modelo y asegurarse de obtener:

    ```
    [{'input_dim': 4, 'output_dim': 6, 'activation': 'relu'},
    {'input_dim': 6, 'output_dim': 8, 'activation': 'relu'},
    {'input_dim': 8, 'output_dim': 10, 'activation': 'relu'},
    {'input_dim': 10, 'output_dim': 3, 'activation': 'softmax'}]
    ```

    Dibujar la red en papel.



In [ ]:
redPrueba = RedNeuronal()
redPrueba.add(Capa(neuronas=6))
redPrueba.add(Capa(neuronas=8))
redPrueba.add(Capa(neuronas=10))
redPrueba.add(Capa(neuronas=3))
redPrueba._compile(X)

In [ ]:
redPrueba.arquitectura

2. Inicializar los pesos de la red del punto anterior (`_init_weights(datos)`) y verificar que los pesos tienen dimensión correcta:

    ```
    capa 0: w=(4, 6) - b=(1, 6)
    capa 1: w=(6, 8) - b=(1, 8)
    capa 2: w=(8, 10) - b=(1, 10)
    capa 3: w=(10, 3) - b=(1, 3)
    ```

  Definir las matrices que se corresponden con las capas de manera que una pasada pueda ser interpretada como el producto de todas ellas. Recordar que en cada paso por cada capa estaremos computando por cada neurona de la capa siguiente:

  $$Z = \sum_{i=1}^{n} X_i \times W_i + b$$

In [ ]:
redPrueba._init_weights(X)

In [ ]:
print('capa 0: w=', redPrueba.pesos[0]['W'].shape, '- b=', redPrueba.pesos[0]['b'].shape)
print('capa 1: w=', redPrueba.pesos[1]['W'].shape, '- b=', redPrueba.pesos[1]['b'].shape)
print('capa 2: w=', redPrueba.pesos[2]['W'].shape, '- b=', redPrueba.pesos[2]['b'].shape)
print('capa 3: w=', redPrueba.pesos[3]['W'].shape, '- b=', redPrueba.pesos[3]['b'].shape)

3. Funciones de activación de una `Capa`:

    1. Verificar que el funcionamiento de `ReLU` se corresponda con:

        ```
        if input > 0:
            return input
        else:
            return 0
        ```

    1. Verificar que el funcionamiento de `softmax` se corresponda con:

  $$\sigma(Z)_i = \frac{e^{z_i}}{\sum_{i=1}^{n} e^{z_j}}$$

    **Nota**: para probar estos dos métodos puede ser util construir un vector de la siguiente manera: `np.array([[1.3, 5.1, -2.2, 0.7, 1.1]])` que genera un vector de tamaño (1,5).

In [ ]:
capa = Capa(1)
prueba = np.array([[1.3, 5.1, -2.2, 0.7, 1.1]])
resultadosReLu = capa.relu(inputs=prueba)
print('resultados obtenidos: ', resultadosReLu)
resultados_esperados = np.array([[1.3, 5.1, 0, 0.7, 1.1]])
print('resultados esperados: ', resultados_esperados)

resultaodsSoftMax = capa.softmax(inputs=prueba)
print('resultados obtenidos: ', np.round(resultaodsSoftMax, 3))
resultados_esperados_softmax = np.array([[0.021, 0.949, 0.001, 0.012, 0.017]])
print('resultados esperados: ', resultados_esperados_softmax)

4. Avancemos con `_forwardprop(datos)`, si corremos la red inicializada con los datos:

    1. ¿Qué nos tipo de objeto nos devuelve este método?

    1. ¿Qué quiere decir cada uno de los valores?

    1. La primera fila, que se correspondería con la primera observación del dataset, ¿qué resultados nos da?¿qué es más probable: 'setosa', 'versicolor' o 'virginica'?¿qué valor es el real?¿por qué?

In [ ]:
print(redPrueba._forwardprop(X)[0])
print('tipo de objeto: ', type(redPrueba._forwardprop(X)))
print('dimensión de la matriz: ', redPrueba._forwardprop(X).shape)
print(y[0])
print(np.unique(y))

5. Arrancamos a propagar para atrás lo aprendido en la primera pasada. Esto lo realizaremos con el método `_backprop`.

    1. ¿Cómo es la derivada de la función de activación `ReLU`?¿Su código es correcto?

    1. ¿Cuál es la operación matemática que hace la función `backward` de la clase `Capa` en el caso de tener como activación a `relu`?

    1. El método `_backprop` toma 2 parámetros: `predicted` y `actual`. ¿qué debemos pasarle en dicho lugar?

        Si la respuesta no fue: en `predicted` le pasamos el resultado de `_forwardprop(...)` y en `actual` le pasamos `y`.... volver a pensarlo. ;-)

    1. Verificar que los `gradientes` y los `pesos` para cada una de las capas tienen el mismo tamaño.

6. Preparemos por último las funciones necesarias para el entrenamiento. Describir brevemente qué hacen las funciones:

    - `_get_accuracy`
    - `_calculate_loss`
    - `_update`

7. Incluyamos finalmente la función `train` y entrenemos una red con la arquitectura propuesta en el punto 1 por 200 epocas.

    1. ¿Qué valores se imprimen?¿Qué es posible interpretar de ellos?

    1. Graficar el _accuracy_ y la _loss_ que arroja el entramiento en función de las _epochs_. ¿Qué se puede concluir? Probablemente la señal sea ruidosa, por lo que se recomienda hacer un suavizado por ventanas deslizantes.

In [ ]:
red_prueba = RedNeuronal()
red_prueba.add(Capa(neuronas=6))
red_prueba.add(Capa(neuronas=8))
red_prueba.add(Capa(neuronas=10))
red_prueba.add(Capa(neuronas=3))
red_prueba.train(X, y, 200)

# graficar el accuracy y la loss por epochs

In [ ]:
window_size = 10  # You can adjust the window size as needed

plt.figure(figsize=(12, 6))

# Plot smoothed accuracy
plt.subplot(1, 2, 1)
plt.plot(np.convolve(red_prueba.accuracy, np.ones(window_size)/window_size, mode='valid'))
plt.title('Smoothed Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

# Plot smoothed loss
plt.subplot(1, 2, 2)
plt.plot(np.convolve(red_prueba.loss, np.ones(window_size)/window_size, mode='valid'))
plt.title('Smoothed Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()

1. Reimplementar la clase `RedNeuronal` utilizando PyTorch

    Hasta ahora hemos construido nuestra propia red neuronal "desde cero", lo cual nos permitió comprender en profundidad cómo funciona cada componente: inicialización de pesos, funciones de activación, forward y backward propagation, cálculo de loss y accuracy, y actualización de pesos.

    Sin embargo, en proyectos reales y más complejos, utilizamos frameworks como **PyTorch** que abstraen estas tareas, permitiéndonos enfocarnos más en el diseño de la arquitectura y el análisis de los resultados.  

    **Objetivo de este inciso**: recrear la arquitectura y entrenamiento de nuestra red neuronal, pero usando herramientas provistas por PyTorch. Esto implica:

    1. Implementar una clase `RedNeuronalTorch` que herede de `nn.Module` y contenga una red con la misma arquitectura:  
    - Entrada de dimensión 4 (por las características del dataset Iris)
    - Capas ocultas de 6, 8 y 10 nodos respectivamente
    - Capa de salida con 3 nodos y activación `softmax`

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class RedNeuronalTorch(nn.Module):
  def __init__(self, dim_features=4, num_classes=3):
    super().__init__()
    self.hidden1 = nn.Linear(dim_features, 6)
    self.hidden2 = nn.Linear(6, 8)
    self.hidden3 = nn.Linear(8, 10)
    self.output = nn.Linear(10, num_classes)
    self.softmax = nn.Softmax(dim=1)
    self.relu = nn.ReLU()

  def forward(self, x_batch):
    x = self.relu(self.hidden1(x_batch))
    x = self.relu(self.hidden2(x))
    x = self.relu(self.hidden3(x))
    z = self.softmax(self.output(x))
    return z

2. Entrenar esta nueva red por 200 épocas utilizando:
    - Función de pérdida: `nn.CrossEntropyLoss`
    - Optimizador: `torch.optim.SGD`
    - Tasa de aprendizaje: 0.01

In [ ]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long) # Use long for CrossEntropyLoss

red_prueba_pytorch = RedNeuronalTorch()

funcion_perdida = nn.CrossEntropyLoss()
optimizador = torch.optim.SGD(red_prueba_pytorch.parameters(), lr=0.01)

loss_history_torch = []
accuracy_history_torch = []

epochs = 200
for epoch in range(epochs):
    # Forward pass
    outputs = red_prueba_pytorch(X_tensor)
    loss = funcion_perdida(outputs, y_tensor)

    # Backward and optimize
    optimizador.zero_grad()
    loss.backward()
    optimizador.step()

    # Calculate accuracy
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_tensor).sum().item() / y_tensor.size(0)

    loss_history_torch.append(loss.item())
    accuracy_history_torch.append(accuracy)

    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

3. Comparar los resultados obtenidos con los del entrenamiento anterior (implementación manual). Algunas preguntas a responder:
    - ¿La convergencia es más rápida o más lenta?
    - ¿Cómo se comporta la pérdida durante el entrenamiento?
    - ¿Cuál implementación fue más fácil de modificar o extender?

4. Graficar la evolución de la **pérdida** y el **accuracy** durante las épocas para ambas implementaciones (manual y PyTorch), idealmente en la misma figura para facilitar la comparación. Podés aplicar una media móvil para suavizar la señal.

    > 💡 **Sugerencia pedagógica**: antes de realizar este inciso, se recomienda repasar los notebooks `9a` y `9b`, donde se presentan una introducción a los tensores y al workflow de ML usando PyTorch.


Crédito: este ejercicio se base en la propuesta de Joe Sasson publicada en [Towards Data Science](https://towardsdatascience.com/coding-a-neural-network-from-scratch-in-numpy-31f04e4d605).

In [ ]:
window_size = 10

plt.figure(figsize=(12, 6))

# Plot smoothed accuracy for both implementations
plt.subplot(1, 2, 1)
plt.plot(np.convolve(red_prueba.accuracy, np.ones(window_size)/window_size, mode='valid'), label='Manual')
plt.plot(np.convolve(accuracy_history_torch, np.ones(window_size)/window_size, mode='valid'), label='PyTorch')
plt.title('Smoothed Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot smoothed loss for both implementations
plt.subplot(1, 2, 2)
plt.plot(np.convolve(red_prueba.loss, np.ones(window_size)/window_size, mode='valid'), label='Manual')
plt.plot(np.convolve(loss_history_torch, np.ones(window_size)/window_size, mode='valid'), label='PyTorch')
plt.title('Smoothed Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

### Código completo (Implementación con Numpy)


In [ ]:
class Capa:
    def __init__(self, neuronas):
        self.neuronas = neuronas

    def forward(self, inputs, weights, bias, activation):
        """
        #Forward Propagation de la capa
        """
        Z_curr = np.dot(inputs, weights.T) + bias

        if activation == 'relu':
            A_curr = self.relu(inputs=Z_curr)
        elif activation == 'softmax':
            A_curr = self.softmax(inputs=Z_curr)

        return A_curr, Z_curr

    def relu(self, inputs):
        """
        #ReLU: función de activación
        """

        return np.maximum(0, inputs)

    def softmax(self, inputs):
        """
        #Softmax: función de activación
        """
        exp_scores = np.exp(inputs)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        return probs

    def backward(self, dA_curr, W_curr, Z_curr, A_prev, activation):
        """
        #Backward Propagation de la capa
        """
        if activation == 'softmax':
            dW = np.dot(A_prev.T, dA_curr)
            db = np.sum(dA_curr, axis=0, keepdims=True)
            dA = np.dot(dA_curr, W_curr)
        else:
            dZ = self.relu_derivative(dA_curr, Z_curr)
            dW = np.dot(A_prev.T, dZ)
            db = np.sum(dZ, axis=0, keepdims=True)
            dA = np.dot(dZ, W_curr)

        return dA, dW, db

    def relu_derivative(self, dA, Z):
        """
        #ReLU: gradiente de ReLU
        """
        dZ = np.array(dA, copy = True)
        dZ[Z <= 0] = 0
        return dZ


In [ ]:
class RedNeuronal:
    def __init__(self, learning_rate=0.01):
        self.red = [] ## capas
        self.arquitectura = [] ## mapeo de entradas -> salidas
        self.pesos = [] ## W, b
        self.memoria = [] ## Z, A
        self.gradientes = [] ## dW, db
        self.lr = learning_rate

    def add(self, capa):
        """
        #Agregar capa a la red
        """
        self.red.append(capa)

    def _compile(self, data):
        """
        #Inicializar la arquitectura
        """
        for idx, _ in enumerate(self.red):
            if idx == 0:
                self.arquitectura.append({'input_dim': data.shape[1],
                                        'output_dim': self.red[idx].neuronas,
                                        'activation':'relu'})
            elif idx > 0 and idx < len(self.red)-1:
                self.arquitectura.append({'input_dim': self.red[idx-1].neuronas,
                                        'output_dim': self.red[idx].neuronas,
                                        'activation':'relu'})
            else:
                self.arquitectura.append({'input_dim': self.red[idx-1].neuronas,
                                        'output_dim': self.red[idx].neuronas,
                                        'activation':'softmax'})
        return self

    def _init_weights(self, data):
        """
        #Inicializar arquitectura y los pesos
        """
        self._compile(data)

        np.random.seed(99)

        for i in range(len(self.arquitectura)):
            self.pesos.append({
                'W':np.random.uniform(low=-1, high=1,
                        size=(self.arquitectura[i]['input_dim'],
                            self.arquitectura[i]['output_dim']
                            )),
                'b':np.zeros((1, self.arquitectura[i]['output_dim']))})

        return self

    def _forwardprop(self, data):
        """
        #Pasada forward completa por la red
        """
        A_curr = data

        for i in range(len(self.pesos)):
            A_prev = A_curr
            A_curr, Z_curr = self.red[i].forward(inputs=A_prev,
                                                    weights=self.pesos[i]['W'].T,
                                                    bias=self.pesos[i]['b'],
                                                    activation=self.arquitectura[i]['activation'])

            self.memoria.append({'inputs':A_prev, 'Z':Z_curr})

        return A_curr

    def _backprop(self, predicted, actual):
        """
        #Pasada backward completa por la red
        """
        num_samples = len(actual)

        ## compute the gradient on predictions
        dscores = predicted
        dscores[range(num_samples),actual] -= 1
        dscores /= num_samples

        dA_prev = dscores

        for idx, layer in reversed(list(enumerate(self.red))):
            dA_curr = dA_prev

            A_prev = self.memoria[idx]['inputs']
            Z_curr = self.memoria[idx]['Z']
            W_curr = self.pesos[idx]['W']

            activation = self.arquitectura[idx]['activation']

            dA_prev, dW_curr, db_curr = layer.backward(dA_curr, W_curr.T, Z_curr, A_prev, activation)

            self.gradientes.append({'dW':dW_curr, 'db':db_curr})

        self.gradientes = list(reversed(self.gradientes))  # Reverse the gradients list

    def _update(self):
        """
        #Actualizar el modelo --> lr * gradiente
        """
        lr = self.lr
        for idx, layer in enumerate(self.red):
            self.pesos[idx]['W'] -= lr * self.gradientes[idx]['dW']
            self.pesos[idx]['b'] -= lr * self.gradientes[idx]['db']

    def _get_accuracy(self, predicted, actual):
        """
        #Calcular accuracy después de cada iteración
        """
        return np.mean(np.argmax(predicted, axis=1)==actual)

    def _calculate_loss(self, predicted, actual):
        """
        #Calculate cross-entropy loss after each iteration
        """
        samples = len(actual)

        correct_logprobs = -np.log(predicted[range(samples),actual])
        data_loss = np.sum(correct_logprobs)/samples

        return data_loss

    def train(self, X_train, y_train, epochs):
        """
        #Entrenar el modelo Stochastic Gradient Descent
        """
        self.loss = []
        self.accuracy = []

        self._init_weights(X_train)

        for i in range(epochs):
            yhat = self._forwardprop(X_train)
            self.accuracy.append(self._get_accuracy(predicted=yhat, actual=y_train))
            self.loss.append(self._calculate_loss(predicted=yhat, actual=y_train))

            self._backprop(predicted=yhat, actual=y_train)

            self._update()

            if i % 20 == 0:
                s = 'EPOCH: {}, ACCURACY: {}, LOSS: {}'.format(i, self.accuracy[-1], self.loss[-1])
                print(s)

        return (self.accuracy, self.loss)